<a href="https://colab.research.google.com/github/Mephisto9727/trabajo-grupal-git/blob/main/prueba_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Celda 1: Configurar entorno
!pip install earthengine-api -q
!pip install geemap -q  # Librería visualización

import ee
import geemap
import folium
import numpy as np
import pandas as pd

print("✅ Librerías instaladas")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.8 MB/s eta 0:00:00
✅ Librerías instaladas


In [3]:
# Celda 2: Autenticar (SOLO PRIMERA VEZ por sesión)
import ee

# Esto abrirá un enlace para autorizar
ee.Authenticate()

# Luego inicializar
ee.Initialize(project='hip-runner-415819')  # Tu proyecto
print("✅ Earth Engine autenticado")

✅ Earth Engine autenticado


In [4]:
# Celda 3: Procesar imágenes de toda una provincia
import ee
import geemap
import time

# Definir región grande (ej: Región Metropolitana Santiago)
region = ee.Geometry.Rectangle([-71.7, -34, -69.8, -32.8])

# 1. CARGAR DATOS MASIVOS (Sentinel-2, 10m resolución)
print("📡 Cargando imágenes Sentinel-2...")
coleccion = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

print(f"📊 Imágenes encontradas: {coleccion.size().getInfo()}")

# 2. COMPOSICIÓN ANUAL (reducir 365 imágenes a 1)
print("🔄 Creando composición anual...")
composicion = coleccion.median().clip(region)

# 3. CALCULAR MÚLTIPLES ÍNDICES (procesamiento paralelo)
print("🧮 Calculando índices espectrales...")

# NDVI - Vegetación
ndvi = composicion.normalizedDifference(['B8', 'B4']).rename('NDVI')

# NDWI - Agua
ndwi = composicion.normalizedDifference(['B3', 'B8']).rename('NDWI')

# NDBI - Construido
ndbi = composicion.normalizedDifference(['B11', 'B8']).rename('NDBI')

# MSAVI2 - Vegetación ajustada
msavi2 = composicion.expression(
    '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED))) / 2',
    {'NIR': composicion.select('B8'), 'RED': composicion.select('B4')}
).rename('MSAVI2')

# 4. APILAR TODOS LOS RESULTADOS
resultados = ee.Image.cat([ndvi, ndwi, ndbi, msavi2])

print("✅ Procesamiento completado")

📡 Cargando imágenes Sentinel-2...
📊 Imágenes encontradas: 486
🔄 Creando composición anual...
🧮 Calculando índices espectrales...
✅ Procesamiento completado


In [8]:
# Celda 4: Análisis de serie temporal pesada - VERSIÓN CORREGIDA
import ee
import pandas as pd
from datetime import datetime

# Región de análisis
punto = ee.Geometry.Point([-70.65, -33.45])  # Santiago centro
area = punto.buffer(5000)  # 5km buffer

# Función para procesar cada imagen - MEJORADA
def calcular_estadisticas(imagen):
    """Calcula NDVI y agrega propiedades de fecha"""

    # Calcular NDVI
    ndvi = imagen.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Obtener estadísticas
    stats = ndvi.reduceRegion(
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        ),
        geometry=area,
        scale=10,
        bestEffort=True,
        maxPixels=1e9
    )

    # Agregar fecha en formato legible
    fecha_milis = imagen.get('system:time_start')
    fecha_formateada = ee.Date(fecha_milis).format('YYYY-MM-dd')

    # Retornar imagen con todas las propiedades
    return imagen.setMulti(stats) \
                .set('fecha_milis', fecha_milis) \
                .set('fecha', fecha_formateada) \
                .set('nubes', imagen.get('CLOUDY_PIXEL_PERCENTAGE'))

# Función principal CORREGIDA
def procesar_serie_temporal(año_inicio, año_fin, nubes_max=30):
    """Procesa serie temporal grande por lotes - VERSIÓN ESTABLE"""

    resultados = []

    for año in range(año_inicio, año_fin + 1):
        print(f"📅 Procesando año {año}...")

        # Filtrar por año con menos nubes
        coleccion_anual = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(area) \
            .filterDate(f'{año}-01-01', f'{año}-12-31') \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', nubes_max))

        # Verificar si hay imágenes
        conteo = coleccion_anual.size().getInfo()
        if conteo == 0:
            print(f"   ⚠️  No hay imágenes con <{nubes_max}% nubes para {año}")
            continue

        print(f"   → Encontradas {conteo} imágenes")

        # Procesar cada imagen
        coleccion_procesada = coleccion_anual.map(calcular_estadisticas)

        try:
            # Obtener información (limitamos a 100 imágenes por año para no saturar)
            info = coleccion_procesada.limit(100).getInfo()

            # Extraer resultados
            for feature in info['features']:
                props = feature['properties']

                fecha = props.get('fecha', 'Fecha no disponible')
                ndvi_mean = props.get('NDVI_mean', None)
                ndvi_std = props.get('NDVI_stdDev', None)
                nubes = props.get('nubes', None)

                if ndvi_mean is not None:
                    resultados.append({
                        'fecha': fecha,
                        'año': año,
                        'mes': int(fecha.split('-')[1]) if '-' in fecha else 0,
                        'dia': int(fecha.split('-')[2]) if '-' in fecha else 0,
                        'ndvi_promedio': round(ndvi_mean, 4),
                        'ndvi_desviacion': round(ndvi_std, 4) if ndvi_std else None,
                        'nubes_porcentaje': nubes
                    })

            print(f"   → {len([r for r in resultados if r['año'] == año])} imágenes procesadas")

        except ee.EEException as e:
            print(f"   ❌ Error procesando año {año}: {e}")
            continue
        except Exception as e:
            print(f"   ⚠️  Error inesperado en año {año}: {e}")
            continue

    return pd.DataFrame(resultados)

# Ejecutar con manejo de errores
print("🚀 Iniciando procesamiento de serie temporal (2019-2023)...")
print("⚠️  Esto puede tomar algunos minutos...")

try:
    df_ndvi = procesar_serie_temporal(2019, 2023, nubes_max=20)

    if not df_ndvi.empty:
        print(f"\n✅ Procesamiento completado!")
        print(f"📊 Total de observaciones: {len(df_ndvi)}")
        print(f"📅 Rango de fechas: {df_ndvi['fecha'].min()} a {df_ndvi['fecha'].max()}")
        print(f"📈 NDVI promedio general: {df_ndvi['ndvi_promedio'].mean():.4f}")

        # Mostrar primeras filas
        print("\n🔍 Primeras 5 observaciones:")
        print(df_ndvi.head())

        # Estadísticas por año
        print("\n📅 Estadísticas por año:")
        stats_anual = df_ndvi.groupby('año').agg({
            'ndvi_promedio': ['count', 'mean', 'min', 'max']
        }).round(4)
        print(stats_anual)

    else:
        print("❌ No se encontraron datos. Intenta aumentar el límite de nubes.")

except Exception as e:
    print(f"❌ Error en el procesamiento principal: {e}")

🚀 Iniciando procesamiento de serie temporal (2019-2023)...
⚠️  Esto puede tomar algunos minutos...
📅 Procesando año 2019...
   → Encontradas 79 imágenes
   → 79 imágenes procesadas
📅 Procesando año 2020...
   → Encontradas 91 imágenes
   → 91 imágenes procesadas
📅 Procesando año 2021...
   → Encontradas 87 imágenes
   → 87 imágenes procesadas
📅 Procesando año 2022...
   → Encontradas 102 imágenes
   → 100 imágenes procesadas
📅 Procesando año 2023...
   → Encontradas 85 imágenes
   → 85 imágenes procesadas

✅ Procesamiento completado!
📊 Total de observaciones: 442
📅 Rango de fechas: 2019-01-04 a 2023-12-29
📈 NDVI promedio general: 0.1641

🔍 Primeras 5 observaciones:
        fecha   año  mes  dia  ndvi_promedio  ndvi_desviacion  \
0  2019-01-04  2019    1    4         0.1855           0.1642   
1  2019-01-04  2019    1    4         0.1952           0.1690   
2  2019-01-19  2019    1   19         0.1774           0.1704   
3  2019-01-19  2019    1   19         0.1809           0.1660   
4

In [11]:
# Celda 5: Clasificación de uso de suelo con Random Forest - VERSIÓN CORREGIDA
import ee
import geemap
import numpy as np

print("🌍 Iniciando clasificación de uso de suelo...")

# 1. DEFINIR REGIÓN MÁS PEQUEÑA Y REALISTA
region = ee.Geometry.Rectangle([-70.68, -33.48, -70.62, -33.43])  # ~6km x 5km = 30km²
print(f"🗺️  Región de estudio: {region.area().getInfo() / 1e6:.2f} km²")

# 2. CARGAR IMAGEN CON MEJORES BANDAS
print("📡 Cargando imagen Sentinel-2...")
imagen = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(region) \
    .filterDate('2023-01-01', '2023-03-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median() \
    .clip(region)

# Calcular índices espectrales
print("🧮 Calculando índices espectrales...")
ndvi = imagen.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndwi = imagen.normalizedDifference(['B3', 'B8']).rename('NDWI')
ndbi = imagen.normalizedDifference(['B11', 'B8']).rename('NDBI')

# Combinar bandas
bandas_originales = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
imagen_completa = imagen.select(bandas_originales) \
    .addBands(ndvi) \
    .addBands(ndwi) \
    .addBands(ndbi)

bandas_todas = bandas_originales + ['NDVI', 'NDWI', 'NDBI']
print(f"📊 Bandas disponibles: {bandas_todas}")

# 3. CREAR PUNTOS DE ENTRENAMIENTO REALISTAS
print("📍 Generando puntos de entrenamiento...")

# Crear polígonos de entrenamiento
urbano = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Rectangle([-70.66, -33.44, -70.65, -33.43]), {'class': 0, 'landcover': 'Urbano'}),
    ee.Feature(ee.Geometry.Rectangle([-70.67, -33.45, -70.66, -33.44]), {'class': 0, 'landcover': 'Urbano'}),
])

vegetacion = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Rectangle([-70.63, -33.46, -70.62, -33.45]), {'class': 1, 'landcover': 'Vegetación'}),
    ee.Feature(ee.Geometry.Rectangle([-70.64, -33.47, -70.63, -33.46]), {'class': 1, 'landcover': 'Vegetación'}),
])

agua = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Rectangle([-70.65, -33.47, -70.64, -33.46]), {'class': 2, 'landcover': 'Agua'}),
    ee.Feature(ee.Geometry.Rectangle([-70.66, -33.47, -70.65, -33.46]), {'class': 2, 'landcover': 'Agua'}),
])

suelo = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Rectangle([-70.68, -33.48, -70.67, -33.47]), {'class': 3, 'landcover': 'Suelo'}),
    ee.Feature(ee.Geometry.Rectangle([-70.67, -33.48, -70.66, -33.47]), {'class': 3, 'landcover': 'Suelo'}),
])

# Combinar todo
entrenamiento_poligonos = urbano.merge(vegetacion).merge(agua).merge(suelo)

# Muestrear puntos dentro de los polígonos
entrenamiento = imagen_completa.sampleRegions(
    collection=entrenamiento_poligonos,
    properties=['class', 'landcover'],
    scale=10,
    geometries=False
)

print(f"📊 Muestras de entrenamiento: {entrenamiento.size().getInfo()}")

# 4. DIVIDIR EN ENTRENAMIENTO Y VALIDACIÓN
print("🎯 Dividiendo datos...")
entrenamiento_mezclado = entrenamiento.randomColumn(seed=42)

train_data = entrenamiento_mezclado.filter(ee.Filter.lt('random', 0.7))
valid_data = entrenamiento_mezclado.filter(ee.Filter.gte('random', 0.7))

print(f"   Entrenamiento: {train_data.size().getInfo()} muestras")
print(f"   Validación: {valid_data.size().getInfo()} muestras")

# 5. ENTRENAR CLASSIFICADOR
print("🤖 Entrenando Random Forest...")
classifier = ee.Classifier.smileRandomForest(
    numberOfTrees=50,  # Reducido para velocidad
    variablesPerSplit=3,
    minLeafPopulation=5,
    seed=42
).train(
    features=train_data,
    classProperty='class',
    inputProperties=bandas_todas
)

# 6. VALIDAR EL MODELO - VERSIÓN CORREGIDA (usar .getInfo())
print("📊 Validando modelo...")
validacion = valid_data.classify(classifier)

# Matriz de confusión - CORREGIDO
matriz_confusion = validacion.errorMatrix('class', 'classification')

# Obtener valores numéricos ANTES de formatear
exactitud = matriz_confusion.accuracy().getInfo()
kappa = matriz_confusion.kappa().getInfo()
precisión = matriz_confusion.consumersAccuracy().getInfo()
sensibilidad = matriz_confusion.producersAccuracy().getInfo()

print("✅ Matriz de confusión:")
print(f"   Exactitud general: {exactitud:.3f}")
print(f"   Índice Kappa: {kappa:.3f}")
print(f"   Precisión por clase: {precisión}")
print(f"   Sensibilidad por clase: {sensibilidad}")

# 7. APLICAR CLASSIFICACIÓN
print("🌍 Aplicando clasificación a toda la región...")
clasificada = imagen_completa.classify(classifier)

# 8. CALCULAR ESTADÍSTICAS - VERSIÓN SIMPLIFICADA
print("📈 Calculando estadísticas de cobertura...")
area_total = region.area().getInfo() / 1e6  # km²

# Contar píxeles por clase
def contar_pixeles(clase_id):
    mascara = clasificada.eq(clase_id)  # Crear máscara para la clase
    conteo = mascara.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=10,
        maxPixels=1e9
    ).getInfo().get('classification', 0)
    return conteo

clases = {
    0: 'Urbano',
    1: 'Vegetación',
    2: 'Agua',
    3: 'Suelo'
}

print("\n📊 DISTRIBUCIÓN DE COBERTURA:")
print("-" * 40)

total_pixeles = 0
resultados = []

for clase_id, clase_nombre in clases.items():
    pixeles = contar_pixeles(clase_id)
    total_pixeles += pixeles
    resultados.append((clase_nombre, pixeles))

# Calcular porcentajes
for clase_nombre, pixeles in resultados:
    if total_pixeles > 0:
        porcentaje = (pixeles / total_pixeles) * 100
        area_km2 = (pixeles * 100) / 1e6  # Cada píxel: 100m²
        print(f"{clase_nombre:12} → {area_km2:6.1f} km² ({porcentaje:5.1f}%)")
    else:
        print(f"{clase_nombre:12} → 0.0 km² (0.0%)")

print("-" * 40)
print(f"Total área: {area_total:.1f} km²")

# 9. VISUALIZAR RESULTADOS (opcional)
try:
    print("\n🎨 Generando visualización...")

    # Configuración de visualización
    vis_params_rgb = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000
    }

    vis_params_clasif = {
        'min': 0,
        'max': 3,
        'palette': ['FF0000', '00FF00', '0000FF', 'FFFF00']  # Rojo, Verde, Azul, Amarillo
    }

    # Mostrar en notebook si estamos en Colab
    try:
        from google.colab import output
        output.enable_custom_widget_manager()

        Map = geemap.Map()
        Map.centerObject(region, 13)
        Map.addLayer(imagen, vis_params_rgb, 'Imagen RGB')
        Map.addLayer(clasificada, vis_params_clasif, 'Clasificación')
        Map.addLayer(entrenamiento_poligonos, {'color': 'white'}, 'Entrenamiento')
        display(Map)

    except:
        # Si no es Colab, guardar HTML
        Map = geemap.Map()
        Map.centerObject(region, 13)
        Map.addLayer(imagen, vis_params_rgb, 'Imagen RGB')
        Map.addLayer(clasificada, vis_params_clasif, 'Clasificación')
        Map.addLayer(entrenamiento_poligonos, {'color': 'white'}, 'Entrenamiento')
        Map.save('clasificacion_santiago.html')
        print("💾 Mapa guardado como 'clasificacion_santiago.html'")

except Exception as e:
    print(f"⚠️  No se pudo generar mapa: {e}")

print("\n✅ Clasificación completada exitosamente!")

🌍 Iniciando clasificación de uso de suelo...
🗺️  Región de estudio: 30.95 km²
📡 Cargando imagen Sentinel-2...
🧮 Calculando índices espectrales...
📊 Bandas disponibles: ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'NDVI', 'NDWI', 'NDBI']
📍 Generando puntos de entrenamiento...
📊 Muestras de entrenamiento: 99234
🎯 Dividiendo datos...
   Entrenamiento: 69378 muestras
   Validación: 29856 muestras
🤖 Entrenando Random Forest...
📊 Validando modelo...
✅ Matriz de confusión:
   Exactitud general: 0.554
   Índice Kappa: 0.405
   Precisión por clase: [[0.5883597883597883, 0.470273631840796, 0.5718860070445085, 0.5929228591648974]]
   Sensibilidad por clase: [[0.6615547329455315], [0.507176391683434], [0.48296376419686315], [0.5629619674774896]]
🌍 Aplicando clasificación a toda la región...
📈 Calculando estadísticas de cobertura...

📊 DISTRIBUCIÓN DE COBERTURA:
----------------------------------------
Urbano       →   12.0 km² ( 32.2%)
Vegetación   →    9.5 km² ( 25.4%)
Agua         →    7.7 km² ( 20.8%

Map(center=[-33.45500120981439, -70.64999999999138], controls=(WidgetControl(options=['position', 'transparent…


✅ Clasificación completada exitosamente!
